# Agent 1: Time Keeper

In this multi-agent system, **`TimeKeeper`** is a precision-driven AI agent designed to provide accurate and real-time time information whenever needed. 

In [1]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

True

## Define user tools

In [ ]:
import json
import datetime
from typing import Any, Callable, Set, Optional


def fetch_current_datetime(format: Optional[str] = None) -> str:
    """
    Get the current time as a JSON string, optionally formatted.

    :param format (Optional[str]): The format in which to return the current time. Defaults to None, which uses a standard format.
    
    :return: The current time in JSON format.
    
    :rtype: str
    """
    current_time = datetime.datetime.now()

    # Use the provided format if available, else use a default format
    if format:
        time_format = format
    else:
        time_format = "%Y-%m-%d %H:%M:%S"

    time_json = json.dumps(
        {"current_time": current_time.strftime(time_format)})
    return time_json


# Example usage
result = fetch_current_datetime()
print(result)

## Create Agent

In [3]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

In [ ]:
from typing import Any, Set, Callable
from azure.ai.projects.models import FunctionTool

user_functions: Set[Callable[..., Any]] = {fetch_current_datetime}
functions = FunctionTool(functions=user_functions)
print(functions.definitions)

[{'type': 'function', 'function': {'name': 'fetch_current_datetime', 'description': 'Get the current time as a JSON string, optionally formatted.', 'parameters': {'type': 'object', 'properties': {'format': {'type': ['string', 'null'], 'description': 'The format in which to return the current time. Defaults to None, which uses a standard format.'}}, 'required': []}}}]


TimeKeeper 角色定義:
<pre>
TimeKeeper：專業時間管理 AI
TimeKeeper 是一款專門提供準確、即時時間資訊的 AI 代理。其主要職責是作為可靠的時間來源，確保使用者與系統能夠隨時獲取精確到秒的當前時間資訊。
TimeKeeper 可根據不同需求調整時間格式，以適應各種使用場景與技術要求。

核心功能
1. 提供當前時間
根據使用者指定的時區或系統預設時區，提供當前時間。

確保時間精確到秒，不會有時間誤差。

2. 格式化時間輸出
支持多種時間格式，包括但不限於：

12 小時制（例如："2:30 PM"）。

24 小時制（例如："14:30"）。

ISO 8601 標準格式（例如："2023-10-12T14:30:00Z"）。

Unix 時間戳（例如："1697111400"）。

預設格式：如果沒有指定格式，則使用標準 24 小時制並帶有日期（例如："2023-10-12 14:30:00"）。

3. 時區自訂
可提供任何指定時區的時間資訊（例如："UTC"、"EST"、"PST"）。

若未指定時區，則預設使用系統本地時區。

4. 錯誤處理
若輸入無效的時區、格式或計算請求，TimeKeeper 會提供清晰的錯誤訊息或替代建議，以確保使用者獲取正確資訊。

目標
TimeKeeper 的最終目標是提供 準確、可靠且符合使用者需求的時間資訊。不論是技術應用還是日常使用，
TimeKeeper 都能確保時間數據的 清晰、精確且靈活可調。
<pre>

In [5]:
instruction = """
# Role
You are TimeKeeper, a specialized AI agent designed to provide accurate and real-time time information. Your primary responsibility is to serve as a reliable and precise source for current time data, ensuring users and systems can access the exact time whenever needed. You are adaptable and capable of formatting the time output to suit various user needs or technical requirements.

# Tasks
1. **Provide Current Time**:
    - Always return the current time based on the user's specified timezone or system default timezone.
    - Ensure the time is accurate to the second.

2. **Format Time Output**:
    - Support multiple time formats based on user requests, such as:
        - 12-hour format (e.g., "2:30 PM").
        - 24-hour format (e.g., "14:30").
        - ISO 8601 format (e.g., "2023-10-12T14:30:00Z").
        - Unix timestamp (e.g., "1697111400").
    - If no format is specified, default to a standard 24-hour format with the date (e.g., "2023-10-12 14:30:00").

3. **Timezone Customization**:
    - Provide time information for any specified timezone (e.g., "UTC", "EST", "PST").
    - If no timezone is specified, default to the system's local timezone.

4. **Error Handling**:
    - Handle invalid timezones, formats, or calculation requests gracefully by providing clear error messages or alternative suggestions.

# Goal
Your ultimate goal is to provide **accurate, reliable, and user-friendly time information** for any request. Ensure clarity, precision, and adaptability in your responses to meet the varying needs of users and systems.
"""

time_keeper = project_client.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="time_keeper",
    description="An AI agent specialized in providing accurate and real-time time information.",
    instructions=instruction,
    tools=functions.definitions,
    # Parameters
    temperature=0.7,
    top_p=0.95,
    # Metadata
    metadata={"group": "internet_threat_analysis"},
)

print(f"Created agent, agent ID: {time_keeper.id}")

Created agent, agent ID: asst_Vjl7RotoXkMazGZug6MTsylS


In [ ]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput


agent_id = time_keeper.id


thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Hello, what time is it now?"
)
print(f"Created message, ID: {message.id}")

# Create and process assistant run in thread with tools
run = project_client.agents.create_run(
    thread_id=thread.id, assistant_id=agent_id)
print(f"Created run, ID: {run.id}")

# Continuously check the status of the run until it is completed or requires action
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)  # Wait for 1 second before checking the status again
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    # If the run requires action and the action is to submit tool outputs
    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            # If no tool calls are provided, cancel the run
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(
                thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        # Process each tool call
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    # Execute the tool call using the defined functions
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    # Handle any errors during tool call execution
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            # Submit the tool outputs back to the run
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )

    # Print the current status of the run
    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")


Created thread, ID: thread_Hj4ndKUGgCsktP5RZi3tS8fo
Created message, ID: msg_VXEpcPrN9gSTKOJ9LVV3LZmF
Created run, ID: run_N5rwAEJp2By432bYS7OCK1kD
Executing tool call: {'id': 'call_xuDISrc1aIly6G1Rsccs5LvT', 'type': 'function', 'function': {'name': 'fetch_current_datetime', 'arguments': '{}'}}
Tool outputs: [{'tool_call_id': 'call_xuDISrc1aIly6G1Rsccs5LvT', 'output': '{"current_time": "2025-04-02 10:31:23"}'}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED


In [7]:
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))

# Conversation
___
### **User** (2025-04-02 10:31:21 台北標準時間)
Hello, what time is it now?
___
### **Assistant** (2025-04-02 10:31:26 台北標準時間)
The current time is **2025-04-02 10:31:23**.
___

In [8]:
messages

{'object': 'list', 'data': [{'id': 'msg_rd9jGLG1dNAiHiuB3sXJ8fbK', 'object': 'thread.message', 'created_at': 1743561086, 'assistant_id': 'asst_Vjl7RotoXkMazGZug6MTsylS', 'thread_id': 'thread_Hj4ndKUGgCsktP5RZi3tS8fo', 'run_id': 'run_N5rwAEJp2By432bYS7OCK1kD', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The current time is **2025-04-02 10:31:23**.', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_VXEpcPrN9gSTKOJ9LVV3LZmF', 'object': 'thread.message', 'created_at': 1743561081, 'assistant_id': None, 'thread_id': 'thread_Hj4ndKUGgCsktP5RZi3tS8fo', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, what time is it now?', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id': 'msg_rd9jGLG1dNAiHiuB3sXJ8fbK', 'last_id': 'msg_VXEpcPrN9gSTKOJ9LVV3LZmF', 'has_more': False}